__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

8815


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

8815


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
7418,2302265468,Участок 10 сот. (ИЖС),5000000,,"с. Широкое, ул. Клюшкина, 47",Продаёться участок 10 соток. Мы собственники. Рядом уже построены дома. Участок без застроек. Документы под Ижс. Ограничений нет. Торг возможен.,16,2022-09-22 09:57:15.623,sevastopol,10.00,True,5.00,0.50,8-20
8033,2560542338,"Участок 6,1 сот. (ИЖС)",1700000,,"пос. Солнечный, Апрельская ул.","Участок 6,1 сотка Ижс.\n\nУчасток 6,1 сотка (Ижс) расположен в г. Севастополь, Нахимовский район, поселок Солнечный, улица Апрельская. Участок ровный, правильной формы, размерами 20м*30м, без уклона. Подъезд асфальтированный. \n\nПозвоните нам, чтобы узнать вс",7,2022-09-26 11:22:23.724,sevastopol,6.10,True,1.70,0.28,4-8
7007,2474508431,Участок 5 сот. (ИЖС),1450000,,"Дачный пер., 10","Продам земельный участок в Севастополе, в районе Лабораторного шоссе. Ижс. Район активно застраивается новенькими котеджами. Площадь участка 5 соток, электричество 15 квт, скоро будет городская вода, газ в перспективе. Среди сосен и холмов открывается шика",8,2022-09-22 09:57:15.623,sevastopol,5.00,True,1.45,0.29,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8815 entries, 0 to 8814
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            8815 non-null   int64         
 1   title               8815 non-null   object        
 2   price               8815 non-null   int64         
 3   obj_name            8815 non-null   object        
 4   adr                 8815 non-null   object        
 5   description         8815 non-null   object        
 6   avito_page          8815 non-null   int64         
 7   ts                  8815 non-null   datetime64[ns]
 8   place               8815 non-null   object        
 9   area                8815 non-null   float64       
 10  is_IJS              8815 non-null   bool          
 11  priceM              8815 non-null   float64       
 12  priceMU             8815 non-null   float64       
 13  area_size_category  8815 non-null   category    

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

8815


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/8815 [00:00<?, ?it/s]

  0%|          | 0/8815 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
30,"ул. Абазы, 4","Севастополь,ул. Абазы, 4"
1845,"6-я Равелинная ул., 22","Севастополь,6-я Равелинная ул., 22"
460,"Нахимовский муниципальный округ, посёлок Любимовка","Севастополь,Нахимовский район , Любимовка"
7053,"Камышовое ш., 24/2","Севастополь,Камышовое ш., 24/2"
4198,"с. Угловое, ул. Гунько, 86","Севастополь,с. Угловое, ул. Гунько, 86"
3940,"Предместная ул., 25","Севастополь,Предместная ул., 25"
5070,Коралловая ул.,"Севастополь,Коралловая ул."
6815,с. Орловка,"Севастополь,с. Орловка"
3293,пос. городского типа Кача,"Севастополь,пос. городского типа Кача"
6094,"ул. 1-я Линия, 41","Севастополь,ул. 1-я Линия, 41"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-09-26 11:27:08 | LocationUpdater: 6081 addresses in index
[INFO    ] 2022-09-26 11:27:08 | LocationUpdater: 6098 addresses total
[INFO    ] 2022-09-26 11:27:08 | LocationUpdater: 6081 addresses defined
[INFO    ] 2022-09-26 11:27:08 | LocationUpdater: 17 addresses undefined


  0%|          | 0/17 [00:00<?, ?it/s]

[INFO    ] 2022-09-26 11:27:14 | LocationUpdater: 17 new addresses found


In [21]:
print( len(loc) )
loc.sample(3)

6098


,latitude,longitude,adr
3922,44.567409,33.557906,"Севастополь,3-я Комендорская ул., 5"
2896,44.569541,33.463044,"Севастополь, район Гагаринский, 5-й, улица Комбрига Потапова, 24"
437,44.60,33.51,"Севастополь,6-я Бастионная улица , 42"


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 8815
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8815 entries, 0 to 8814
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               8815 non-null   object        
 1   price               8815 non-null   int64         
 2   obj_name            8815 non-null   object        
 3   adr_orig            8815 non-null   object        
 4   description         8815 non-null   object        
 5   avito_page          8815 non-null   int64         
 6   ts                  8815 non-null   datetime64[ns]
 7   place               8815 non-null   object        
 8   area                8815 non-null   float64       
 9   is_IJS              8815 non-null   bool          
 10  priceM              8815 non-null   float64       
 11  priceMU             8815 non-null   float64       
 12  area_size_category  8815 non-null   category      
 13  url 